In [ ]:
import re

In [ ]:
with open("input-example.txt") as f:
    lines = f.readlines()
lines = [l.rstrip() for l in lines]
lines[:5]

In [ ]:
max_x = 0
for l in lines[:-2]:
    max_x = max(max_x, len(l))
max_y = len(lines[:-2])
print(max_y, max_x)


padded_lines = []
for l in lines[:-2]:
    padded_lines.append(l.ljust(max_x))
# padded_lines[:5]

start_pos = (-1, -1)
for i in range(0, len(padded_lines[0])):
    if padded_lines[0][i] == ".":
        start_pos = (0, i)
        break

start_pos

In [ ]:
directions = [n for n in re.findall(r"(\d+|[R|L])", lines[-1])]
directions[:5]

In [ ]:
def print_walked_fields(walked_fields, padded_lines):
    for i in range(0, len(padded_lines)):
        for j in range(0, len(padded_lines[0])):
            if (i, j) in walked_fields:
                print("O", end="")
            else:
                print(padded_lines[i][j], end="")
        print()


def in_bounds(pos, field):
    return (
        pos[0] >= 0 and pos[0] < len(field) and pos[1] >= 0 and pos[1] < len(field[0])
    )


def get_field(pos, field):
    return field[pos[0]][pos[1]]


def mv_pos(pos, dir):
    return (pos[0] + dir[0], pos[1] + dir[1])


def mv_pos_skip(pos, dir, max_x, max_y, padded_lines):
    next_pos = mv_pos(pos, dir)
    next_pos_mod = (next_pos[0] % max_y, next_pos[1] % max_x)
    while get_field(next_pos_mod, padded_lines) not in [".", "#"]:
        next_pos = mv_pos(next_pos, dir)
        next_pos_mod = (next_pos[0] % max_y, next_pos[1] % max_x)

    return next_pos_mod


# mv_pos_skip((1,11), (0,1), max_x, max_y, padded_lines)

In [ ]:
dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]
dir_index = 0

walk_dirs = directions.copy()
walked_fields = [start_pos]
curr_pos = start_pos
while walk_dirs:
    steps_left = int(walk_dirs.pop(0))
    if walk_dirs:
        upcoming_turn = walk_dirs.pop(0)
    else:
        upcoming_turn = None

    next_potential_field = mv_pos_skip(
        curr_pos, dirs[dir_index], max_x, max_y, padded_lines
    )
    while steps_left > 0 and get_field(next_potential_field, padded_lines) != "#":
        steps_left -= 1
        curr_pos = next_potential_field
        walked_fields.append(curr_pos)

        next_potential_field = mv_pos_skip(
            curr_pos, dirs[dir_index], max_x, max_y, padded_lines
        )

    if upcoming_turn == "L":
        dir_index = (dir_index - 1) % len(dirs)
    elif upcoming_turn == "R":
        dir_index = (dir_index + 1) % len(dirs)

# print(directions[:10])
# print(curr_pos, dir_index)
res = (curr_pos[0] + 1) * 1000 + (curr_pos[1] + 1) * 4 + dir_index
print(res)
# print(walked_fields)
# print_walked_fields(walked_fields, padded_lines)

part 2

In [ ]:
# There is certainly a general solution somehow
# But I chose to go the tedious way of hardcoding every "walking off" to any of the sides that are not connected
# First for the example and then again for the actual cube 
# That was definitely something...

In [ ]:
with open("input-example.txt") as f:
    lines = f.readlines()
lines = [l.rstrip() for l in lines]

max_x = 0
for l in lines[:-2]:
    max_x = max(max_x, len(l))
max_y = len(lines[:-2])

padded_lines = []
for l in lines[:-2]:
    padded_lines.append(l.ljust(max_x))

start_pos = (-1, -1)
for i in range(0, len(padded_lines[0])):
    if padded_lines[0][i] == ".":
        start_pos = (0, i)
        break

directions = [n for n in re.findall(r"(\d+|[R|L])", lines[-1])]

In [ ]:
def mv_pos_cube_example(
    pos,
    dir,
):
    cube_len = 4
    next_pos = mv_pos(pos, dir)
    # walking off to the left
    # at the top
    if dir == (0, -1) and next_pos[0] < cube_len and next_pos[1] == 7:
        next_pos = (cube_len, pos[0] + cube_len)
        return (next_pos, (1, 0))
    elif (
        dir == (0, -1)
        and next_pos[0] >= 1 * cube_len
        and next_pos[0] < 2 * cube_len
        and next_pos[1] == -1
    ):
        next_pos = (11, 5 * cube_len - (pos[0] + 1))
        return (next_pos, (-1, 0))
    elif dir == (0, -1) and next_pos[0] >= 2 * cube_len and next_pos[1] == 7:
        next_pos = (7, 4 * cube_len - (pos[0] + 1))
        return (next_pos, (-1, 0))
    # walking off to the top
    elif (
        dir == (-1, 0)
        and next_pos[0] == 3
        and next_pos[1] >= 0
        and next_pos[1] < 1 * cube_len
    ):
        next_pos = (0, 3 * cube_len - (pos[1] + 1))
        return next_pos, (0, 1)
    elif (
        dir == (-1, 0)
        and next_pos[0] == 3
        and next_pos[1] >= 1 * cube_len
        and next_pos[1] < 2 * cube_len
    ):
        next_pos = (pos[1] - cube_len, 2 * cube_len)
        return next_pos, (0, 1)
    elif (
        dir == (-1, 0)
        and next_pos[0] == -1
        and next_pos[1] >= 2 * cube_len
        and next_pos[1] < 3 * cube_len
    ):
        next_pos = (5, 3*cube_len - (pos[1]+1))
        return next_pos, (1, 0)
    elif (
        dir == (-1, 0)
        and next_pos[0] == 7
        and next_pos[1] >= 3 * cube_len
    ):
        next_pos = (5*cube_len- (pos[1]+1), 11)
        return next_pos, (0, -1)
    # walking off to the right
    if dir == (0, 1) and next_pos[0] < 1 * cube_len and next_pos[1] == 12 :
        next_pos = (3*cube_len-(pos[0]+1), 15)
        return (next_pos, (0,-1))
    elif (
        dir == (0, 1)
        and next_pos[0] >= 1 * cube_len
        and next_pos[0] < 2 * cube_len
        and next_pos[1] == 12
    ):
        next_pos = (8, 5 * cube_len - (pos[0] + 1))
        return (next_pos, (1, 0))
    elif dir == (0, 1) and next_pos[0] >= 2 * cube_len and next_pos[1] == 16:
        next_pos = (3*cube_len-(pos[0]+1), 11)
        return (next_pos, (0, -1))
    # walking off to the bottom
    elif (
        dir == (1, 0)
        and next_pos[0] == 8
        and next_pos[1] >= 0
        and next_pos[1] < 1 * cube_len
    ):
        next_pos = (11, 3 * cube_len - (pos[1]+1))
        return next_pos, (-1, 0)
    elif (
        dir == (1, 0)
        and next_pos[0] == 8
        and next_pos[1] >= 1 * cube_len
        and next_pos[1] < 2 * cube_len
    ):
        next_pos = (4*cube_len - (pos[1]+1), 2 * cube_len)
        return next_pos, (0, 1)
    elif (
        dir == (1, 0)
        and next_pos[0] == 12
        and next_pos[1] >= 2 * cube_len
        and next_pos[1] < 3 * cube_len
    ):
        next_pos = (7, 3*cube_len - (pos[1]+1))
        return next_pos, (-1, 0)
    elif (
        dir == (1, 0)
        and next_pos[0] == 12
        and next_pos[1] >= 3 * cube_len
    ):
        next_pos = (5*cube_len- (pos[1]+1), 0)
        return next_pos, (0, 1)
    else:
        return (next_pos, dir)




In [ ]:
dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]
dirs_dict = {(0, 1):0, (1, 0):1, (0, -1):2, (-1, 0):3}
dir_index = 0

walk_dirs = directions.copy()
walked_fields = [start_pos]
curr_pos = start_pos
while walk_dirs:
    steps_left = int(walk_dirs.pop(0))
    if walk_dirs:
        upcoming_turn = walk_dirs.pop(0)
    else:
        upcoming_turn = None

    next_potential_field, next_potential_dir = mv_pos_cube_example(
        curr_pos, dirs[dir_index]
    )
    while steps_left > 0 and get_field(next_potential_field, padded_lines) != "#":
        steps_left -= 1
        curr_pos = next_potential_field
        dir_index = dirs_dict[next_potential_dir]
        walked_fields.append(curr_pos)

        next_potential_field, next_potential_dir = mv_pos_cube_example(
            curr_pos, dirs[dir_index]
        )

    if upcoming_turn == "L":
        dir_index = (dir_index - 1) % len(dirs)
    elif upcoming_turn == "R":
        dir_index = (dir_index + 1) % len(dirs)

# print(directions[:10])
# print(curr_pos, dir_index)
res = (curr_pos[0] + 1) * 1000 + (curr_pos[1] + 1) * 4 + dir_index
print(res)
# print(walked_fields)
# print_walked_fields(walked_fields, padded_lines)

In [ ]:
# Example test cases
mv_pos_cube_example((3, 8), (0, -1))  # ((4,7), (1,0))
mv_pos_cube_example((2, 8), (0, -1))  # ((4,6), (1,0))

mv_pos_cube_example((4, 0), (0, -1))  # ((11,15), (-1,0))
mv_pos_cube_example((7, 0), (0, -1))  # ((11,12), (-1,0))

mv_pos_cube_example((8, 8), (0, -1))  # ((7,7), (-1,0))
mv_pos_cube_example((11, 8), (0, -1))  # ((7,6), (-1,0))

mv_pos_cube_example((4, 7), (-1, 0))  # ((3,8), (0,1))

mv_pos_cube_example((4, 0), (-1, 0))  # ((0,11), (1,0))
mv_pos_cube_example((4, 4), (-1, 0))  # ((0,10), (1,0))

mv_pos_cube_example((0, 8), (-1, 0))  # ((5,3), (1,0))
mv_pos_cube_example((0, 9), (-1, 0))  # ((5,2), (1,0))

mv_pos_cube_example((8, 12), (-1, 0))  # ((7,11), (0,-1))
mv_pos_cube_example((8, 13), (-1, 0))  # ((6,11), (0,-1))

mv_pos_cube_example((0, 11), (0, 1))  # ((11,15), (0,-1))
mv_pos_cube_example((1, 11), (0, 1))  # ((10,15), (0,-1))

mv_pos_cube_example((4, 11), (0, 1))  # ((8,15), (1,0))
mv_pos_cube_example((5, 11), (0, 1))  # ((8,14), (1,0))

mv_pos_cube_example((8, 15), (0, 1))  # ((3,11), (0,-1))
mv_pos_cube_example((9, 15), (0, 1))  # ((2,11), (0,-1))

mv_pos_cube_example((7, 0), (1, 0))  # ((11,11), (-1,0))
mv_pos_cube_example((7, 1), (1, 0))  # ((11,10), (-1,0))

mv_pos_cube_example((7, 4), (1, 0))  # ((11,8), (0,1))
mv_pos_cube_example((7, 5), (1, 0))  # ((10,8), (0,1))

mv_pos_cube_example((11, 8), (1, 0))  # ((7,3), (-1,0))
mv_pos_cube_example((11, 9), (1, 0))  # ((7,2), (-1,0))

mv_pos_cube_example((11, 12), (1, 0))  # ((7,0), (0,1))
mv_pos_cube_example((11, 13), (1, 0))  # ((6,0), (0,1))

In [ ]:
# Now the same *fun* exercise again for the real cube 

In [ ]:
with open("input.txt") as f:
    lines = f.readlines()
lines = [l.rstrip() for l in lines]

max_x = 0
for l in lines[:-2]:
    max_x = max(max_x, len(l))
max_y = len(lines[:-2])

padded_lines = []
for l in lines[:-2]:
    padded_lines.append(l.ljust(max_x))

start_pos = (-1, -1)
for i in range(0, len(padded_lines[0])):
    if padded_lines[0][i] == ".":
        start_pos = (0, i)
        break

directions = [n for n in re.findall(r"(\d+|[R|L])", lines[-1])]

In [ ]:
def mv_pos_cube(
    pos,
    dir,
):
    cube_len = 50
    next_pos = mv_pos(pos, dir)
    # walking off to the left
    if dir == (0, -1) and next_pos[0] < cube_len and next_pos[1] == 49:
        next_pos = (3*cube_len-(pos[0]+1), 0)
        return (next_pos, (0, 1))
    elif (
        dir == (0, -1)
        and next_pos[0] >= 1 * cube_len
        and next_pos[0] < 2 * cube_len
        and next_pos[1] == 49
    ):
        next_pos = (100,(pos[0])- cube_len  )
        return (next_pos, (1, 0))
    elif (
        dir == (0, -1)
        and next_pos[0] >= 2 * cube_len
        and next_pos[0] < 3 * cube_len
        and next_pos[1] == -1
    ):
        next_pos = (3 * cube_len - (pos[0] + 1), 50)
        return (next_pos, (0, 1))
    elif dir == (0, -1) and next_pos[0] >= 3 * cube_len and next_pos[1] == -1:
        next_pos = (0, (pos[0])-2*cube_len)
        return (next_pos, (1, 0))
    # walking off to the top
    if dir == (-1, 0) and next_pos[0] == 99 and next_pos[1] < cube_len:
        next_pos = (cube_len+ (pos[1]), 50)
        return (next_pos, (0,1))
    elif (
        dir == (-1, 0)
        and next_pos[0] == -1
        and next_pos[1] >= cube_len
        and next_pos[1] < 2* cube_len
    ):
        next_pos = (pos[1]+2*cube_len, 0)
        return (next_pos, (0, 1))
    elif dir == (-1, 0) and next_pos[0] == -1 and next_pos[1] >= 2*cube_len:
        next_pos = (199, pos[1]- 2*cube_len)
        return (next_pos, (-1,0))
    # walking off to the right
    if dir == (0, 1) and next_pos[0] < cube_len and next_pos[1] == 150:
        next_pos = (3*cube_len-(pos[0]+1), 99)
        return (next_pos, (0, -1))
    elif (
        dir == (0, 1)
        and next_pos[0] >= 1 * cube_len
        and next_pos[0] < 2 * cube_len
        and next_pos[1] == 100
    ):
        next_pos = (49,(pos[0])+ cube_len  )
        return (next_pos, (-1, 0))
    elif (
        dir == (0, 1)
        and next_pos[0] >= 2 * cube_len
        and next_pos[0] < 3 * cube_len
        and next_pos[1] == 100
    ):
        next_pos = (3*cube_len-(pos[0]+1), 149)
        return (next_pos, (0, -1))
    elif dir == (0, 1) and next_pos[0] >= 3 * cube_len and next_pos[1] == 50:
        next_pos = (149,pos[0]-2*cube_len)
        return (next_pos, (-1, 0))
    # walking off to the bottom
    if dir == (1, 0) and next_pos[0] == 200 and next_pos[1] < cube_len:
        next_pos = (0, 2*cube_len+ (pos[1]))
        return (next_pos, (1,0))
    elif (
        dir == (1, 0)
        and next_pos[0] == 150
        and next_pos[1] >= cube_len
        and next_pos[1] < 2* cube_len
    ):
        next_pos = (pos[1]+2*cube_len, 49)
        return (next_pos, (0, -1))
    elif dir == (1, 0) and next_pos[0] == 50 and next_pos[1] >= 2*cube_len:
        next_pos = (pos[1]-cube_len, 99)
        return (next_pos, (0,-1))
    else:
        return (next_pos, dir)


In [ ]:
dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]
dirs_dict = {(0, 1):0, (1, 0):1, (0, -1):2, (-1, 0):3}
dir_index = 0

walk_dirs = directions.copy()
walked_fields = [start_pos]
curr_pos = start_pos
while walk_dirs:
    steps_left = int(walk_dirs.pop(0))
    if walk_dirs:
        upcoming_turn = walk_dirs.pop(0)
    else:
        upcoming_turn = None

    next_potential_field, next_potential_dir = mv_pos_cube(
        curr_pos, dirs[dir_index]
    )
    while steps_left > 0 and get_field(next_potential_field, padded_lines) != "#":
        steps_left -= 1
        curr_pos = next_potential_field
        dir_index = dirs_dict[next_potential_dir]
        walked_fields.append(curr_pos)

        next_potential_field, next_potential_dir = mv_pos_cube(
            curr_pos, dirs[dir_index]
        )

    if upcoming_turn == "L":
        dir_index = (dir_index - 1) % len(dirs)
    elif upcoming_turn == "R":
        dir_index = (dir_index + 1) % len(dirs)

# print(directions[:10])
# print(curr_pos, dir_index)
res = (curr_pos[0] + 1) * 1000 + (curr_pos[1] + 1) * 4 + dir_index
print(res)
#print(walked_fields)
# print_walked_fields(walked_fields, padded_lines)

In [ ]:
# real input test cases
mv_pos_cube((0, 50), (0, -1))  # ((149,0), (0,1))
mv_pos_cube((1, 50), (0, -1))  # ((149,0), (0,1))

mv_pos_cube((50, 50), (0, -1))  # ((100,0), (1,0))
mv_pos_cube((51, 50), (0, -1))  # ((100,1), (1,0))

mv_pos_cube((100, 0), (0, -1))  # ((49,50), (0,1))
mv_pos_cube((101, 0), (0, -1))  # ((48,50), (0,1))

mv_pos_cube((150, 0), (0, -1))  # ((0,50), (1,0))
mv_pos_cube((151, 0), (0, -1))  # ((0,51), (1,0))

mv_pos_cube((100, 0), (-1, 0))  # ((50,50), (0,1))
mv_pos_cube((100, 1), (-1, 0))  # ((51,50), (0,1))

mv_pos_cube((0, 50), (-1, 0))  # ((150,0), (0,1))
mv_pos_cube((0, 51), (-1, 0))  # ((151,0), (0,1))

mv_pos_cube((0, 100), (-1, 0))  # ((199,0), (-1,0))
mv_pos_cube((0, 101), (-1, 0))  # ((199,1), (-1,0))

mv_pos_cube((0, 149), (0, 1))  # ((149,99), (0,-1))
mv_pos_cube((1, 149), (0, 1))  # ((148,99), (0,-1))

mv_pos_cube((50, 99), (0, 1))  # ((49,100), (-1,0))
mv_pos_cube((51, 99), (0, 1))  # ((49,101), (-1,0))

mv_pos_cube((100, 99), (0, 1))  # ((49,149), (0,-1))
mv_pos_cube((101, 99), (0, 1))  # ((48,149), (0,-1))

mv_pos_cube((150, 49), (0, 1))  # ((149,50), (-1,0))
mv_pos_cube((151, 49), (0, 1))  # ((149,51), (-1,0))

mv_pos_cube((199, 0), (1,0))  # ((0,100), (1,0))
mv_pos_cube((199, 1), (1,0))  # ((0,101), (1,0))

mv_pos_cube((149, 50), (1,0))  # ((150,49), (0,-1))
mv_pos_cube((149, 51), (1,0))  # ((151,49), (0,-1))

mv_pos_cube((49, 100), (1,0))  # ((50,99), (0,-1))
mv_pos_cube((49, 101), (1,0))  # ((51,99), (0,-1))